In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.utils import shuffle
from tqdm import tqdm
from sklearn.utils import shuffle
import math
import sys
import os

In [2]:
dataFrame_full_artist = pd.read_excel("full_music_data.xlsx")
dataFrame_full_artist.sort_values(by=["artists_id", 'year'], ascending=True)


,artist_names,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,popularity,year,release_date,song_title (censored),Unnamed: 19
30809,['Emarosa'],[1000177],0.386,0.941,0.483,90.913,-2.168,1,6,0.000550,0.000000,0.1400,0.0515,0,273653,48,2008,7/8/2008,The Past Should Stay Dead,NaN
30808,['Emarosa'],[1000177],0.334,0.924,0.359,170.159,-3.805,0,8,0.004440,0.000000,0.1360,0.2760,0,228987,47,2010,6/28/2010,A Toast to the future kids!,NaN
5106,"['3OH!3', 'Kesha']","[1000302, 80358]",0.682,0.889,0.827,138.021,-4.166,1,0,0.005640,0.000000,0.3600,0.0804,0,192440,63,2010,6/25/2010,My First Kiss (feat. Ke$ha),NaN
5108,['3OH!3'],[1000302],0.590,0.813,0.517,161.868,-3.543,0,5,0.004160,0.000000,0.7480,0.3420,1,199600,45,2008,7/7/2008,RICHMAN,NaN
5109,['3OH!3'],[1000302],0.851,0.633,0.513,139.944,-5.052,1,10,0.000905,0.000003,0.1160,0.1100,0,185533,45,2008,7/7/2008,STARSTRUKK,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10882,['Beach House'],[998969],0.509,0.790,0.625,147.073,-7.105,0,0,0.218000,0.125000,0.1440,0.0293,0,320467,68,2015,8/28/2015,***** Song,NaN
10890,['Beach House'],[998969],0.490,0.464,0.142,104.896,-7.478,1,1,0.381000,0.894000,0.0555,0.0284,0,368547,55,2015,8/28/2015,PPP,NaN
13526,"['Bingo Players', 'Far East Movement']","[999790, 870646]",0.802,0.981,0.804,127.974,-2.708,1,7,0.020400,0.000004,0.2560,0.0658,0,166760,51,2013,1/1/2013,Get Up (Rattle),NaN
13527,['Bingo Players'],[999790],0.744,0.957,0.529,127.991,-4.800,1,7,0.001520,0.884000,0.3520,0.0519,0,287184,59,2011,10/31/2011,Rattle,NaN


In [3]:
# read_dictionary
name_dic = open("name_to_id_dictionary.txt", "r")
string = ""
line = 1
while line:
    line = name_dic.readline()
    string += line
name_dic = eval(string)
person_num = len(name_dic)
print("person_num = ",person_num)

person_num =  5641


In [4]:
for idx, row in tqdm(dataFrame_full_artist.iterrows(),):
    m = row["artists_id"].find(',')
    if m != -1:
        Id=int(row["artists_id"][1:m])
    else:
        Id=int(row["artists_id"][1:-1])
    dataFrame_full_artist["artists_id"][idx] = Id
dataFrame_full_artist

0it [00:00, ?it/s]<ipython-input-4-31702c7ee6ec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame_full_artist["artists_id"][idx] = Id
98340it [01:21, 1209.71it/s]


,artist_names,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,popularity,year,release_date,song_title (censored),Unnamed: 19
0,"[""Fat Freddy's Drop""]",178301,0.600,0.365,0.131,130.046,-13.083,0,9,0.06720,0.585000,0.0921,0.0498,0,437200,54,2005,2005,Ernie,NaN
1,"[""Fat Freddy's Drop""]",178301,0.874,0.326,0.179,119.620,-13.302,0,11,0.01360,0.148000,0.0993,0.1310,0,581008,53,2005,2005,Wandering Eye,NaN
2,"[""Fat Freddy's Drop""]",178301,0.670,0.531,0.336,139.385,-8.267,0,9,0.01560,0.345000,0.3060,0.0377,0,431293,55,2009,8/7/2009,The Raft,NaN
3,"[""Alexander O'Neal""]",625201,0.761,0.702,0.850,104.773,-8.523,1,7,0.10800,0.000031,0.0935,0.0389,0,304427,34,1991,1/1/1991,All True Man,NaN
4,"[""Alexander O'Neal""]",625201,0.661,0.828,0.902,115.078,-12.673,0,11,0.27600,0.000001,0.2870,0.0390,0,264933,37,1987,7/29/1987,(What Can I Say) To Make You Love Me,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98335,['ZZ Top'],690254,0.276,0.892,0.715,80.475,-7.035,1,11,0.40900,0.000000,0.7140,0.0893,0,115973,33,1975,4/18/1975,Jailhouse Rock - **** Remaster,NaN
98336,['ZZ Top'],690254,0.700,0.592,0.906,109.847,-10.434,1,11,0.10800,0.018400,0.1180,0.0681,0,263627,32,1976,11/29/1976,It's Only Love,NaN
98337,['ZZ Top'],690254,0.709,0.709,0.863,111.544,-12.023,1,2,0.11800,0.000011,0.1250,0.0370,0,158400,31,1987,1987,Balinese,NaN
98338,['ZZ Top'],690254,0.552,0.651,0.533,161.548,-10.624,0,4,0.00494,0.034900,0.1320,0.2290,0,232533,43,1992,4/13/1992,La Grange - **** Remaster,NaN


In [5]:
full_music_data = {"danceability":[0.5263203874313606,0.16403205144827973],
                    "energy":[0.5342478371079926,0.26447724025183267],
                    "valence":[0.5332812944885094,0.2585376133654879],
                    "tempo":[118.96853772625585,29.925255322835987],
                    "loudness":[-10.755944010575554,5.067612837357291],
                    "acousticness":[0.4190568774831198,0.35370999027336875],
                    "instrumentalness":[0.1255745396816148,0.2724918312399652],
                    "liveness":[0.20700085794183445,0.18610339608517842],
                    "speechiness":[0.06380584502745576,0.07610388534934469],
                    "explicit":[0.037085621313809235,0.18897163280656168],
                    "duration_ms":[238590.92141549726,108823.17603973577],
                    "mode":[0.7219951189749847,0.4480158112737567],
                    "popularity":[35.69332926581249,17.262527298425432],}
print(len(full_music_data))

13


In [6]:
extractedData = pd.read_csv("extractedData.csv")
extractedData

,Unnamed: 0,artists_id,danceability,energy,valence,tempo,loudness,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,mode,popularity,year,key
0,77723,74,0.315058,1.250588,-0.179012,-0.061972,0.728932,-1.110110,-0.454049,0.848986,-0.233968,5.09555,-1.086569,0.620525,26,1983,9
1,68873,335,-0.355543,0.104932,-0.534859,0.270222,-1.010151,-1.156193,-0.459689,-0.445993,-0.260247,-0.19625,-0.438307,0.620525,33,1992,1
2,68876,335,-0.629879,-0.034966,0.416646,1.090031,-1.153809,-1.171516,-0.449425,-0.558834,-0.343029,-0.19625,-0.371161,0.620525,31,1992,11
3,68874,335,-0.623783,0.437664,0.849852,0.241885,-1.071916,-1.183268,2.669531,-0.515847,0.048541,-0.19625,-0.872277,0.620525,38,1993,7
4,68878,335,0.967370,-0.859234,1.155417,0.335017,-1.651479,-1.128769,-0.447076,-0.531967,0.076135,-0.19625,-0.388556,-1.611539,41,1993,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98335,60481,3661296,0.717418,1.156062,1.190228,-0.666077,1.173717,-0.726745,-0.460838,-0.790425,-0.289155,-0.19625,-0.348987,0.620525,53,2012,8
98336,60483,3661296,1.613585,0.898951,-0.221559,0.234700,1.389401,-1.019357,-0.459950,-0.695317,-0.320691,-0.19625,-0.199019,0.620525,54,2012,10
98337,60480,3661296,0.851539,1.042631,1.074191,-0.666512,1.219893,-0.755016,-0.460838,-0.782903,-0.236596,-0.19625,-0.352663,0.620525,58,2013,8
98338,73261,3661738,-0.215326,-0.484911,-0.983537,0.704504,-0.503799,1.133536,0.438272,-0.456740,-0.473114,-0.19625,0.472134,0.620525,58,2015,5


In [7]:
artist_number_df = extractedData["artists_id"].value_counts()
art_number = artist_number_df.values
index = artist_number_df.index.values
print("art_number = ", art_number)  # 对应id的数目
print("index = ", index)  # 艺术家的id
artist_dict = {}
for i in range(len(art_number)):
    artist_dict[index[i]] = art_number[i]

index_sorted = np.sort(index)
artist_num =[]
for i in range(len(index_sorted)):
    artist_num.append([index_sorted[i],artist_dict[index_sorted[i]]])
artist_num = np.array(artist_num).T[-1].astype(int)
artist_position = np.cumsum(artist_num)
print("artist_num = \n", artist_num)
print("artist_position = \n", artist_position)

art_number =  [938 692 684 ...   1   1   1]
index =  [ 26350 792507  79016 ... 562144 830375  26637]
artist_num = 
 [ 1 16 11 ...  9  1  1]
artist_position = 
 [    1    17    28 ... 98338 98339 98340]


In [8]:
# 获得位置范围的range列表
pos = 0
range_list = []
for i in range(len(artist_position)):
    range_list.append([pos, artist_position[i]])
    pos = artist_position[i]
print(len(range_list))
# print(range_list)

5641


In [14]:
for _range_ in tqdm(range_list):
    for i in range(_range_[0],_range_[1]):
        extractedData.loc[i,"num"] = _range_[1] - _range_[0]
extractedData = extractedData[extractedData["num"] > 100]

  0%|                                                                                         | 0/5641 [00:00<?, ?it/s]<ipython-input-14-3aa0520f849a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extractedData.loc[i,"num"] = _range_[1] - _range_[0]
100%|██████████████████████████████████████████████████████████████████████████████| 5641/5641 [03:03<00:00, 30.76it/s]


In [15]:
extractedData.to_csv("very_important_artist_data.csv")
extractedData

,Unnamed: 0,artists_id,danceability,energy,valence,tempo,loudness,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,mode,popularity,year,key,num
251,7896.0,3595.0,-0.733518,-1.260025,-0.879103,-1.525118,-1.045671,0.749041,-0.460833,-0.548087,-0.416613,-0.19625,-0.137139,0.620525,0.0,1939.0,2.0,102.0
252,7913.0,3595.0,-1.916213,-1.475544,-1.320045,-1.315161,-1.543933,1.091129,-0.460232,-0.069858,-0.381135,-0.19625,0.592117,0.620525,0.0,1939.0,1.0,102.0
253,7883.0,3595.0,-1.910117,-0.847891,-1.416743,-0.859392,-0.132421,1.605675,2.618154,0.870479,-0.339087,-0.19625,-0.543781,0.620525,0.0,1952.0,3.0,102.0
254,7881.0,3595.0,-1.684551,-1.562508,-1.049291,-0.313365,-1.246357,1.540650,-0.457799,-0.413753,-0.350913,-0.19625,-0.276209,0.620525,32.0,1980.0,0.0,102.0
255,7884.0,3595.0,0.065107,-0.205113,-0.755330,-0.417759,-0.181753,0.867782,-0.460478,-0.118218,-0.055265,-0.19625,-0.546978,0.620525,38.0,1980.0,2.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96820,51446.0,2784799.0,-0.532337,-0.462224,-1.370328,0.132479,0.524891,1.057203,-0.460838,-0.860279,-0.408729,-0.19625,0.618518,-1.611539,68.0,2019.0,9.0,103.0
96821,51465.0,2784799.0,0.747900,0.014187,-0.144201,0.869181,-0.086837,-0.133038,-0.437644,-0.601283,-0.156442,5.09555,-0.334468,-1.611539,76.0,2019.0,7.0,103.0
96822,51482.0,2784799.0,-1.879635,-1.207090,-1.528912,-1.411134,-0.342184,1.549131,-0.150003,-0.602895,-0.354855,5.09555,0.095045,0.620525,72.0,2019.0,5.0,103.0
96823,51483.0,2784799.0,0.034625,-0.310983,-1.277498,0.365125,0.020117,1.337093,-0.456691,-0.429873,-0.273387,5.09555,-0.186568,-1.611539,73.0,2019.0,9.0,103.0


In [16]:
extractedData["artists_id"].value_counts()

26350.0     938
792507.0    692
79016.0     684
119107.0    610
816890.0    566
           ... 
479028.0    102
3595.0      102
98465.0     101
478603.0    101
170879.0    101
Name: artists_id, Length: 164, dtype: int64

In [13]:
# read_dictionary
name_dic = open("name_to_id_dictionary.txt", "r")
string = ""
line = 1
while line:
    line = name_dic.readline()
    string += line
name_dic = eval(string)
person_num = len(name_dic)
print("person_num = ",person_num)

person_num =  5641
